# On Calibration of Modern Neural Networks

- toc: true 
- badges: true
- comments: true
- categories: [neural-networks]

While working on deep learning projects, I observed that confidence scores of deep learning models (for classification tasks) were extremely high while the model performed well on metrics like validation accuracy, AUROC, etc. I realized that this was a problem because the confidence score of the model on any out-of-distribution data was over 90%. This was a problem of miscalibration and this post will be an attempt to summarise this [paper](https://arxiv.org/abs/1706.04599) which tries to understand the topic of calibration in detail.

### Introduction to Calibration

Calibration refers to probability estimates of a deep neural network being representative of the true correctness likelihood. For example, given 100 predictions each with a confidence estimate of 0.8, 80 of them should be correctly classified. Therefore, for a perfectly calibrated model, the average accuracy should match the average confidence estimate 

While modern neural networks have become more accurate, they have poor calibration. The Figure from the paper compares a 5-layer LeNet (LeCun et al., 1998) with a 110 layer ResNet (He et al., 2016) on CIFAR-100 dataset. Based on the definition of calibration, it is essential that average accuracy of a model matches the average confidence in the best case situation. While LeNet has the average confidence close to the accuracy, the average confidence of a ResNet is much higher than the accuracy. 

<img src="calibration/average-conf.png" alt="reliability-diag" style="zoom: 30%;" />

This is an issue because an uncalibrated model is incapable of distinguishing samples between in-distribution and out-of-distribution samples. For example, DNNs trained to classify MNIST images often produce high confident probability 91% even for random noise (see the work of (Hendrycks & Gimpel, 2016)).

#### Methods To Quantify and Visualize Calibration

The following methods are used visualize and quantify calibration.

1. Reliability diagrams
2. Expected Calibration Error (ECE)
3. Maximum Calibration Error (MCE)

We’ll broadly go over the definitions and the code that can be used to implement these methods

##### Reliability Diagrams : Visual Representation of Model Calibration
<img src="calibration/rel-diag.png" alt="reliability-diag" style="zoom: 30%;" />

A reliability diagram plots the expected accuracy as a function of confidence. For a finite set of samples, the confidences can be grouped into $M$ bins (each of size $1/M$), and the average accuracy for each group can be calculated accordingly.  Let $B_m$ be the set of indices whose predictions falls in the interval $I_m = (\frac{m-1}{M}, \frac{m}{M})$, the expected accuracy for that interval would be,
$$
acc(B_m) = \frac{1}{|B_m|}\sum_{i \epsilon B_m}1(\hat{y_i} = y_i)
$$
For a perfectly calibrated model, the expected accuracy for each bin would plot the identity function. Any deviation from the identity function would represent miscalibration.   

##### Expected Calibration Error (ECE)

> One notion of miscalibration is the difference in expectation between confidence and accuracy

Expected Calibration Error is a scalar summary statistic of calibration. The mathematical form of it is as follows,
$$
E_{\hat{p}}[|P(\hat{Y} = Y | \hat{P} = p) - p|]
$$
An emperical form can be approximated by partitioning predicted confidences into M equally spaced bins (similar to reliability diagrams) and taking a weighted average of the bins’ accuracy/confidence difference.
$$
ECE = \sum_{i=m}^{M}\frac{|B_m|}{n}|acc(B_m) - conf(B_m)|
$$

##### Maximum Calibration Error (MCE)

For high risk applications where reliable probability estimates are needed, the maximum calibration error is calculated. It is the maximum deviation between bin accuracy and confidence. The exact mathematical form, 
$$
\max_{p \epsilon [0, 1]}|P(\hat{Y} = Y | \hat{P} = p) - p|
$$
An emperical form can be approximated similarly to $(3)$, 
$$
ECE = \max_{m\epsilon{\{1,...,M\}}}|acc(B_m) - conf(B_m)|
$$
**Note:** We can visualize MCE and ECE on reliability diagrams. MCE is the largest calibration gap (red bars) across all bins, whereas ECE is a weighted average of all gaps. For perfectly calibrated classifiers, MCE and ECE both equal 0.

### Temperature Scaling

The main contribution of this paper is the technique, Temperature Scaling which is a post training approach to calibrate predictions of a model. We’ll go through some concepts before discussing the exact method which will make it easier to digest temperature scaling.

#### Softmax Function with Temperature

The softmax function ($\sigma_{SM}$​) usually used as an activation function to go from logits to probabilistic outputs. For instance, for a classification problem involving more than $K > 2$ Classes, the network produces an output logit $z_i$, class prediction $\hat{y_i}$ and confidence score $\hat{p_i}$ for each input $\boldsymbol{x_i}$. The network logits $z_i$ are vectors, where $\hat{y_i} = argmax_k \space z_i^{k}$, and $\hat{p_i}$ is derived using the softmax function $\sigma_{SM}$. (The temperature term $T$ is usually set to 1).
$$
\sigma_{SM}(\boldsymbol{z_i})^{(k)}  = \frac{\exp(z_i^{(k)}/ T)}{\sum_{j=1}^{K}\exp(z_i^{(j)}/ T)}
$$
Varying the temperature term can make the softmax outputs smoother (higher $T$) or peaky (lower $T$). If $T \rightarrow \infty $, the softmax function becomes a uniform distribution with the output being $1/K$, whereas if $T \rightarrow 0 $, the output becomes a point mass.

#### Intuition of the method proposed (Temperature Scaling)

The intuition of the method proposed in this paper is as follows. If our model confidence scores are too high (in other words peaky), then it might mean that moving towards a temperature scaling value ($T$) that is higher (reducing the peakiness) might help calibrate the model predictions. 

#### Platt Scaling 

This is a parametric approach introduced in paper (Platt et al. 1999) where the non-probabilistic predictions (logit) of a binary classifier are used as features for a logistic regression model, which is trained on the validation set to return probabilities. Basically, Platt Scaling learns scalar parameters $a, \space b \space\large\epsilon \space \mathbb{R}$ and outputs $\hat{q_i}=\sigma(az_i + b)$ as the calibrated probability. The loss used is the negative log likelihood (NLL) to tune the scalar parameters.

#### Temperature Scaling (Extension of Platt Scaling)

This is a simple single parameter extension of Platt Scaling where we aim to learn the temperature paramter $T > 0$ for all the classes in the softmax function. Given the logit vector $z_i$, the calibrated output now becomes,
$$
\hat{q_i} =\max_{k} \frac{\exp(z_i^{(k)}/ T)}{\sum_{j=1}^{K}\exp(z_i^{(j)}/ T)}
$$
where T is optimized with respect to NLL on the validation set. As temperature scaling does not affect the maximum of the softmax, the class prediction $\hat{y_i}$ and accuracy of the model should remain the same.



**Why Logistic Regression?**

The reason for choosing to fine-tune with a logistic regression model is that, the outputs of a logistic regressor is calibrated. To understand why this is true, let’s first look at the logistic regression setting.

The loss function for logistic regression is the standard negative log likelihood function

$$
J(\theta) = -\frac{1}{m}\sum_{i=1}^{M}[y^{(i)}\log(h_{\theta}(x^{(i)}))\space + \space (1 - y^{(i)})\log{(1 - h_{\theta}(x^{(i)}))}]
$$

Where we get the output by passing the $\theta^Tx^{(i)}$ through a sigmoid function ($\alpha$ is the bias term),

$$
h_{\theta}(x^{(i)}) = \frac{1}{1 + \exp(-\alpha -\sum_j\theta_{j}x_j^{(i)})}
$$

Since we want to minimize the loss function with respect to $\theta$, we take a partial derivative with respect to $\theta$, 

$$
\begin{aligned}
\frac{\partial{J(\theta)}}{\partial{\theta}} & =  -\frac{1}{m}\sum_{i=1}^{M}[\frac{y^{(i)}}{h_{\theta}(x^{(i)})} - \frac{(1 - y^{(i)})}{1 - h_{\theta}(x^{(i)})}]\frac{\partial{h_{\theta}(x^{(i)})}}{\partial{\theta}} \\
& =  -\frac{1}{m}\sum_{i=1}^{M}[\frac{y^{(i)} - h_{\theta}(x^{(i)})}{h_{\theta}(x^{(i)})(1 - h_{\theta}(x^{(i)}))}] \frac{\partial{h_{\theta}(x^{(i)})}}{\partial{\theta}} 
\end{aligned}
$$


If we take a partial derivative of the softmax output $h_{\theta}(x^{(i)})$,

$$
\begin{aligned}
\frac{\partial{h_{\theta}(x^{(i)})}}{\partial{\theta}} & = -(\frac{1}{1 + \exp(-\alpha -\sum_j\theta_{j}x_j^{(i)})})^2 * \exp(-\alpha -\sum_j\theta_{j}x_j^{(i)}) * \frac{\partial(-\alpha-\sum_j\theta_{j}x_j^{(i)})}{\partial{\theta}}\\
& = h_{\theta}(x^{(i)})(1 - h_{\theta}(x^{(i)})) * \frac{\partial(-\alpha -\sum_j\theta_{j}x_j^{(i)})}{\partial{\theta}}
\end{aligned}
$$

If we replace the above in the $\frac{\partial{J(\theta)}}{\partial{\theta}} $, equation,

$$
\frac{\partial{J(\theta)}}{\partial{\theta}} = -\frac{1}{m}\sum_{i=1}^{M}(y^{(i)} - h_{\theta}(x^{(i)})\frac{\partial(-\alpha -\sum_j\theta_{j}x_j^{(i)})}{\partial{\theta}}
$$

For a fully trained model, as $\frac{\partial{J(\theta)}}{\partial{\theta_j}}$  would be $0$ for each $\theta_{j}$, if we look at $\theta_{0} = \alpha$ and set $\frac{\partial{J(\theta)}}{\partial{\theta_0}} = 0$, 

$$
\frac{\partial{J(\theta)}}{\partial{\theta_j}} = -\frac{1}{m}\sum_{i=1}^{M}(y^{(i)} - h_{\theta}(x^{(i)}) = 0
$$
$$
\sum_{i=1}^{M}h_{\theta}(x^{(i)}) = \sum_{i=1}^{M}y^{(i)}
$$

The above expression means that the average sigmoid outputs ($\sum_{i=1}^{M}h_{\theta}(x^{(i)})$) will be equal to accuracy of the model which satisfies the calibration condition.


#### Entropy Maximization Perspective: Why Temperature Scaling helps increase Entropy?

Our usual training process for a classification problem involves minimizing the cross entropy loss using the predicted probaility $\hat{p_i}$ and the one-hot labels $y_i$. While minimizing this loss, we bring the predicted distribution $\hat{p_i}$ closer to the one-hot true label vector $y_i$ (as minimizing cross entropy is equivalent to minizing KL divergence). 

The entropy of the predicted probability ($H(\hat{p_i}) = -\hat{p_i}\space log(\hat{p_i})$) is 0 when $\hat{p_i}$ perfectly matches $y_i$ while it is maximum when $\hat{p_i}$ is a uniform distribution. Therefore, temperature scaling will not only help in increasing entropy as we move away from a peaky distribution to a more uniform version of it. Therefore, we can think of this process of temperature scaling/entropy maximization as a form of *regularizationed. 

In the figure below, we can see that effect of calibration on the Validation Set (i.e. optimizing T with respect to NLL). Two points to note, 

- Look at how the blue line indicates the pre-calibration entropy decreasing with epochs (overfitting) and the optimal T increasing (to undo the effect the overfitting). 
- The post-calibration entropy lies **above** the pre-calibration entropy line indicating the effect of Temperature Scaling

<img src="calibration/entropy.png" alt="entropy" style="zoom: 20%;" />